# Match company names to ISIN codes using Azure AI Language and Wiki data

Entity linking is a natural language processing task that involves identifying and disambiguating entities in a text. Entities are words or phrases that refer to real-world objects, such as people, places, organizations, events, products, etc. Disambiguating entities means resolving the ambiguity that arises when the same word or phrase can refer to different entities, depending on the context. For example, the word "Apple" can refer to the fruit, the company, or the Beatles' record label, depending on the text.

Entity linking is important for many applications, such as information extraction, knowledge graph construction, question answering, text summarization, and sentiment analysis. By linking entities to their unique identifiers in a knowledge base, such as Wiki data, we can enrich the text with additional information and enable semantic search and reasoning.

This example notebook uses the Azure AI Language service and Wiki data to link company names mentioned in unstructured text (e.g. news articles or transcriptions of earning calls) to company ids (ISIN codes). 

The notebook is divided into the following sections:

- Import libraries
- Util functions
- Get ISIN codes from list of wikipedia pages
- Get linked entities from text using Azure AI Language
- Function to get ISIN codes from text
- Get ISIN codes for each document in `data` folder

Please refer to this page for more information on the Azure AI Language entity linking capabilities: https://learn.microsoft.com/en-us/azure/ai-services/language-service/entity-linking/quickstart?tabs=windows&pivots=programming-language-python

## Imports

In [2]:
# Install required packages
%pip install azure-ai-textanalytics==5.2.0
%pip install nltk

import os
import nltk
import requests

from nltk.tokenize import sent_tokenize
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

nltk.download('punkt')

language_key = os.environ.get('LANGUAGE_KEY')
language_endpoint = os.environ.get('LANGUAGE_ENDPOINT')

def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


     ---------------------------------------- 0.0/239.3 kB ? eta -:--:--
     ----------------- -------------------- 112.6/239.3 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 239.3/239.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: azure-ai-textanalytics
    Found existing installation: azure-ai-textanalytics 5.2.1
    Uninstalling azure-ai-textanalytics-5.2.1:
      Successfully uninstalled azure-ai-textanalytics-5.2.1
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: C:\Users\haalkema\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.5 MB 3.0 MB/s eta 0:00:01
     -------- ------------------------------- 0.3/1.5 MB 4.2 MB/s eta 0:00:01
     ------------------------------- -------- 1.2/1.5 MB 8.3 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: C:\Users\haalkema\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haalkema\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Util functions

In [3]:
def get_isin(wikipedia_name):
    redirects = check_redirects(wikipedia_name)
    if redirects is not None:
        wikipedia_name = redirects
    try:
        url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&props=claims&titles={wikipedia_name}&format=json"
        response = requests.get(url)
        data = response.json()

        entity_id = next(iter(data["entities"]))
        value = data["entities"][entity_id]['claims']['P946'][0]['mainsnak']["datavalue"]["value"]
        return value
    except:
        return None


def check_redirects(titles):
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&titles={titles}&redirects"
    response = requests.get(url)
    data = response.json()

    if "query" in data:
        if "redirects" in data["query"]:
            return data["query"]["redirects"][0]["to"]
    else:
        return None

## Get ISIN codes from list of wikipedia pages

In [4]:
companies = ["Apple_Inc.", "Alphabet_Inc.", "ASML_Holding", "Tesla,_Inc.", "Verizon_Communications"]

for company in companies:
    print(get_isin(company))

US0378331005
US02079K3059
NL0010273215
US88160R1014
US92343V1044


## Get linked entities from text

In [5]:
try:
    documents = ["""
With the upcoming launch of Apple Vision Pro, we're seeing strong excitement in enterprise, leading organizations across many industries, such as Walmart, Nike, Vanguard, Stryker, Bloomberg and SAP started leveraging and investing in Apple Vision Pro as the new platform to bring innovative spatial computing experiences to their customers and employees. 
"""]
    result = client.recognize_linked_entities(documents = documents)[0]

    print("Linked Entities:\n")
    for entity in result.entities:
        print("\tName: ", entity.name, "\tId: ", entity.data_source_entity_id, "\tUrl: ", entity.url)
        print("\tMatches:")
        for match in entity.matches:
            print("\t\tText:", match.text)
            print("\t\tConfidence Score: {0:.2f}".format(match.confidence_score))
        
except Exception as err:
    print("Encountered exception. {}".format(err))


Linked Entities:

	Name:  Apple Inc. 	Id:  Apple Inc. 	Url:  https://en.wikipedia.org/wiki/Apple_Inc.
	Matches:
		Text: Apple
		Confidence Score: 0.37
		Text: Apple
		Confidence Score: 0.37
	Name:  Visi On 	Id:  Visi On 	Url:  https://en.wikipedia.org/wiki/Visi_On
	Matches:
		Text: Vision
		Confidence Score: 0.10
		Text: Vision
		Confidence Score: 0.10
	Name:  .pro 	Id:  .pro 	Url:  https://en.wikipedia.org/wiki/.pro
	Matches:
		Text: Pro
		Confidence Score: 0.06
		Text: Pro
		Confidence Score: 0.06
	Name:  Walmart 	Id:  Walmart 	Url:  https://en.wikipedia.org/wiki/Walmart
	Matches:
		Text: Walmart
		Confidence Score: 0.30
	Name:  Nike, Inc. 	Id:  Nike, Inc. 	Url:  https://en.wikipedia.org/wiki/Nike,_Inc.
	Matches:
		Text: Nike
		Confidence Score: 0.32
	Name:  The Vanguard Group 	Id:  The Vanguard Group 	Url:  https://en.wikipedia.org/wiki/The_Vanguard_Group
	Matches:
		Text: Vanguard
		Confidence Score: 0.06
	Name:  Stryker 	Id:  Stryker 	Url:  https://en.wikipedia.org/wiki/Stryker
	M

## Function to get ISIN codes from text

In [6]:
def get_isin_codes_from_text(client, text):
    companies = {}

    try:
        sentences = sent_tokenize(text)

        for sentence in sentences:
            results = client.recognize_linked_entities(documents = [sentence])

            for result in results:
                for entity in result.entities:
                    if entity.data_source_entity_id in companies:
                        continue
                    isin = get_isin(entity.data_source_entity_id)
                    if isin is not None:
                        companies.update({entity.data_source_entity_id: isin})
                        print("\tId: ", entity.data_source_entity_id, "\tUrl: ", entity.url, "\tMatches:", [match.text for match in entity.matches], "\tISIN: ", isin)
                        
        return companies
            
    except Exception as err:
        print("Encountered exception. {}".format(err))

## Get ISIN codes for each document in data folder

Make sure to add any news articles or transcriptions of earning calls to a folder called `data` before running the call below.

In [7]:
folder_path = "data"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    with open(file_path, "r") as file:
        print(filename)
        text = file.read()
        get_isin_codes_from_text(client, text)


aws-chips.txt
	Id:  Amazon (company) 	Url:  https://en.wikipedia.org/wiki/Amazon_(company) 	Matches: ['Amazon', 'AMZN'] 	ISIN:  US0231351067
	Id:  Nvidia 	Url:  https://en.wikipedia.org/wiki/Nvidia 	Matches: ['Nvidia'] 	ISIN:  US67066G1040
tesla.txt
	Id:  Tesla, Inc. 	Url:  https://en.wikipedia.org/wiki/Tesla,_Inc. 	Matches: ['Tesla'] 	ISIN:  US88160R1014
	Id:  Morningstar, Inc. 	Url:  https://en.wikipedia.org/wiki/Morningstar,_Inc. 	Matches: ['Morningstar'] 	ISIN:  US6177001095
	Id:  NIO (car company) 	Url:  https://en.wikipedia.org/wiki/NIO_(car_company) 	Matches: ['Nio', 'NIO'] 	ISIN:  US62914V1061
	Id:  Nvidia 	Url:  https://en.wikipedia.org/wiki/Nvidia 	Matches: ['Nvidia'] 	ISIN:  US67066G1040
	Id:  Apple Inc. 	Url:  https://en.wikipedia.org/wiki/Apple_Inc. 	Matches: ['Apple'] 	ISIN:  US0378331005
	Id:  Microsoft 	Url:  https://en.wikipedia.org/wiki/Microsoft 	Matches: ['Microsoft'] 	ISIN:  US5949181045
